In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')
!pip install xmltodict
import xmltodict

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


# Launch DPS for extract_filter_atl08.py
using the rebinned 30m h5 granules

In [2]:
!pip install -U -r /projects/icesat2_boreal/dps/requirements_main.txt
from os import path
import os, glob
import datetime
import pandas as pd
import geopandas as gpd
# !pip install s3fs 
#!pip install pystac
import s3fs

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
dir(s3fs)

In [ ]:
curr_dir = wk_dir = os.path.dirname(os.path.realpath('__file__'))
print(curr_dir)

## Test extract_filter_atl08.py code on Test Data
We are running extract_filter_atl08.py (but *FILTERING* is turned off):

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "https://maap-ops-workspace.s3.amazonaws.com.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python lib/extract_filter_atl08.py -i "path/to/h5file" -o "path/of/out/dir" --no-filter-qual --do_30m

In [25]:
if False:
    ## NOTE: this ATL08 needs to be v005 not v004
    #h5_fn = '/projects/test_data/test_extract_filter_atl08_30m/ATL08_30m_20181014001049_02350102_005_01.h5'
    h5_fn = 'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/09/21/19/04/41/436383/ATL08_30m_20181014031924_02370102_005_01.h5'
    h5_fn = h5_fn.replace('https://maap-ops-workspace.s3.amazonaws.com/lduncanson', '/projects/my-private-bucket')
    #!python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_extract_filter_atl08_30m" -i $h5_fn --no-filter-qual --do_30m --get_gedi_rh
    !python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_extract_filter_atl08_30m" -i $h5_fn --no-filter-qual --do_30m 

# Register DPS algorithm
We need to register a DPS algorithm called `run_extract_filter_atl08_ubuntu` before proceeding to the chunks below...

In [2]:
!python /projects/register-algorithm /projects/Developer/icesat2_boreal/dps/alg_2-3/algorithm_config.yaml

{
  "algorithm_name": "run_extract_filter_atl08",
  "code_version": "master",
  "environment_name": "ubuntu",
  "repo_url": "https://repo.ops.maap-project.org/icesat2_boreal/icesat2_boreal.git",
  "docker_container_url": "mas.maap-project.org:5000/root/ade-base-images/r:latest",
  "queue": "maap-dps-worker-8gb",
  "algorithm_description": null,
  "build_command": "icesat2_boreal/dps/build_command_main.sh",
  "script_command": "icesat2_boreal/dps/alg_2-3/run_extract_filter_atl08.sh",
  "disk_space": "10GB",
  "algorithm_params": [
    {
      "field": "input_file",
      "download": true
    }
  ]
}
{"code": 200, "message": {"id": "b7ae158e6338ae13f06c7c3234025d7bf07878a4", "short_id": "b7ae158e", "created_at": "2022-09-21T16:36:47.000+00:00", "parent_ids": ["f3e7e8fb54228850d5ad3d2690999218eb62baf4"], "title": "Registering algorithm: run_extract_filter_atl08_ubuntu", "message": "Registering algorithm: run_extract_filter_atl08_ubuntu", "author_name": "www-data", "author_email": "www-dat

### Build the input DPS list of granules - s3fs implementation
Find all the processed h5 granules that Eric ran  
Find all the extracted CSV failes that Paul ran  
Match h5 granules with corresponding CSVs  
Find h5 granules that dont yet have csvs

In [35]:
%%time

s3 = s3fs.S3FileSystem()
YEAR = "2022"
MONTH_LIST = ['09','10']
bucket = "s3://maap-ops-workspace"

# Match DPS output from 2 algorithms
# These are the 2 DPS algs that produced the files we're searching for
DPS_ALG_LIST = ['run_rebinning_ubuntu', 'run_extract_filter_atl08_ubuntu']
# These are the 2 file extensions we're searching for
FILE_EXT_LIST = ['.h5', '_30m.csv']

for i, SEARCH_EXT_TYPE in enumerate(FILE_EXT_LIST):
    
    searchkey_type_list = [f"lduncanson/dps_output/{DPS_ALG_LIST[i]}/master/{YEAR}/{MONTH}/{str('%02d' % DAY)}/**/*{SEARCH_EXT_TYPE}" for DAY in range(1,32) for MONTH in MONTH_LIST]
    # Concat list of lists to data frame
    atl0830m_type_path_df = pd.concat([pd.DataFrame(s3.glob(os.path.join(bucket, searchkey)), columns=['maap_path']) for searchkey in searchkey_type_list])
    #Convert data frame to list
    ATL08_type_GRANULE_LIST = [i.replace("maap-ops-workspace", "https://maap-ops-workspace.s3.amazonaws.com") for i in atl0830m_type_path_df.maap_path.to_list()]
    
    grans = [os.path.split(i)[-1].replace(SEARCH_EXT_TYPE,'') for i in atl0830m_type_path_df.maap_path.to_list()]
    
    if '.h5' in SEARCH_EXT_TYPE:
        ATL08_h5_GRANULE_df = pd.DataFrame({'h5_path': ATL08_type_GRANULE_LIST, 'granule_name': grans})
    else:
        ATL08_csv_GRANULE_df = pd.DataFrame({'csv_path': ATL08_type_GRANULE_LIST, 'granule_name': grans})
      
print(f"Count of H5 granules processed in {YEAR} for months {MONTH_LIST}: {len(ATL08_h5_GRANULE_df)}")    
print('Matching extracted csv to ATL08 30m h5 granules..')
merged = ATL08_h5_GRANULE_df.merge(ATL08_csv_GRANULE_df, how='left', on='granule_name')

print('Finding the null matches (the granules that still need extracted csvs...')
INPUT_ATL08_GRANULE_LIST = merged[merged.csv_path.isnull()].h5_path.to_list()
print(f'# of CSV extractions still needed:\t{len(INPUT_ATL08_GRANULE_LIST)}\n')

Count of H5 granules processed in 2022 for months ['09', '10']: 13087
Matching extracted csv to ATL08 30m h5 granules..
Finding the null matches (the granules that still need extracted csvs...
# of CSV extractions still needed:	13087

CPU times: user 11.2 s, sys: 107 ms, total: 11.3 s
Wall time: 29 s


In [102]:
#'ATL08_30m_20200609215801_11510705_005_01_30m.csv' not found
ATL08_csv_GRANULE_df.iloc[2].granule_name

'ATL08_30m_20181020230436_03410105_005_01_30m-checkpoint.csv'

In [93]:
TEST_DPS  = False

if TEST_DPS:
    print('Running DPS on a SUBSET list of input')
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST[-100:]
else:
    if False:
        LIST_SIZE = 15000
        print(f'Running DPS on the {LIST_SIZE} SUBSET list of input')
        DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST[0:LIST_SIZE]
    else:
        print(f'Running DPS on the FULL list of input')
        DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST
    
DPS_INPUT_ATL08_GRANULE_LIST[-1]

Running DPS on the FULL list of input


'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/05/04/47/47/547100/ATL08_30m_20210715133745_03351202_005_01.h5'

## Run a DPS job across the list

In [94]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_ATL08_GRANULE_LIST)

for i, INPUT_ATL08_GRANULE in enumerate(DPS_INPUT_ATL08_GRANULE_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_extract_filter_atl08'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_param_dict = {
                        'input_file': INPUT_ATL08_GRANULE
                    }

    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_ATL08_GRANULE
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] = datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE
    
    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
        print(f"DPS run #: {DPS_num}\t| granule name: {os.path.basename(INPUT_ATL08_GRANULE)}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

DPS run #: 1	| granule name: ATL08_30m_20181014051811_02380106_005_01.h5	| submit status: success	| job id: 3278d457-0512-4aca-aaaa-8e5af0b6d3d4
DPS run #: 100	| granule name: ATL08_30m_20190115121627_02760202_005_01.h5	| submit status: success	| job id: 084a729c-4e3f-4841-80a2-17b4765b2594
DPS run #: 500	| granule name: ATL08_30m_20191114161750_07450502_005_01.h5	| submit status: success	| job id: f9d623e6-14a4-4ba9-a79a-62ed9611477f
DPS run #: 1000	| granule name: ATL08_30m_20210319110333_13051006_005_01.h5	| submit status: success	| job id: 6e8b0073-9931-4894-9d12-8636f7d12647
DPS run #: 1130	| granule name: ATL08_30m_20210715133745_03351202_005_01.h5	| submit status: success	| job id: 69cbaf7f-08d8-4a92-8476-0f9194488384
CPU times: user 13.5 s, sys: 927 ms, total: 14.4 s
Wall time: 4min 34s


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,3278d457-0512-4aca-aaaa-8e5af0b6d3d4,1,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:04:32.015769,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,1044269e-0c48-4be8-9794-a9dacd0b3c59,2,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:04:32.101211,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,d3dc76fb-2d62-4331-be4b-d57c7b23d8e8,3,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:04:32.257089,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,8c495e81-2f4e-4126-be50-a442910ee90c,4,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:04:32.401972,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,71196b4d-b2f4-479c-b356-69954c6702f6,5,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:04:32.616129,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
...,...,...,...,...,...,...,...,...,...,...
0,success,200,14bca5fa-3f76-425b-b337-30698be1b4dc,1126,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:09:04.281418,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,5ac05e78-25cb-42be-b3f0-b43eb42f4552,1127,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:09:04.539713,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,f2e39743-7b5d-407d-807d-06a49c94e1b1,1128,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:09:04.718672,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,f7705cad-fabf-48db-8bae-3f7e179737d2,1129,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-11 03:09:04.933538,3,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb


After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

In [95]:
%%time
def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id', right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	1130
Count pending jobs:	0
Count running jobs:	1
Count succeeded jobs:	1129
Count failed jobs:	0
% of failed jobs:	0.0
CPU times: user 11.8 s, sys: 885 ms, total: 12.7 s
Wall time: 34 s


In [97]:
!python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t 'ATL08' -y 2022 -alg_name 'run_rebinning_ubuntu' 

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:  ATL08

Output dir:  /projects/my-public-bucket/DPS_tile_lists
# of duplicate tiles: 209
Final # of tiles: 46166
df shape :                                              s3_path  ... tile_num
0  s3://maap-ops-workspace/lduncanson/dps_output/...  ...       NA
1  s3://maap-ops-workspace/lduncanson/dps_output/...  ...       NA
2  s3://maap-ops-workspace/lduncanson/dps_output/...  ...       NA
3  s3://maap-ops-workspace/lduncanson/dps_output/...  ...       NA
4  s3://maap-ops-workspace/lduncanson/dps_output/...  ...       NA

[5 rows x 4 columns]
Writing tindex master csv: /projects/my-public-bucket/DPS_tile_lists/ATL08_tindex_master.csv


In [85]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Succeeded'].iloc[2].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            '83d185f4-9df2-4095-b1f1-f5260375362c'),
                           ('wps:Output',
                            OrderedDict([('@id',
                                          'output-2022-03-10T22:36:22.286595'),
                                         ('wps:Data',
                                          ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/lduncanson/dps_output/run_extract_filter_atl08_ubuntu/master/2022/03/10/22/36/22/286595',
                                           '

##### Pick a job and manually test - does it produce output as expected?
if so, then AWS had some issue during the DPS: contact MAAP dev about this

In [12]:
f = fails_df.iloc[1].tile_num.replace('https://maap-ops-workspace.s3.amazonaws.com/lduncanson','/projects/my-private-bucket')
!python /projects/icesat2_boreal/dps/alg_2-3/extract_filter_atl08.py --i $f --no-filter-qual --do_30m -o /projects/my-public-bucket/test_output


Written by:
	Nathan Thomas	| @Nmt28
	Paul Montesano	| paul.m.montesano@nasa.gov

Min lat: 45.0
Max lat: 75.0
Min lon: -180.0
Max lon: 180.0
Month range: 6-9

ATL08 granule name: 	ATL08_30m_20210714235338_03261206_005_01
Input dir: 		/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/03/05/04/51/06/341661

Segment length: 30m
Find src nodata value using max of h_can: 	nan

Building pandas dataframe...
Setting pandas df nodata values to np.nan for some basic eval.
# of ATL08 obs: 		211006
# of ATL08 obs (can pho.>=0): 	77997
# of ATL08 obs (toc pho.>=0): 	77997
# of ATL08 obs (h_can>=0): 	77905
# of ATL08 obs (h_can<0): 	92
Setting out pandas df nodata values: 	3.4028234663852886e+38
Quality Filtering: 	[OFF] (do downstream)
Geographic Filtering: 	[ON] xmin = -180.0, xmax = 180.0, ymin = 45.0, ymax = 75.0
Creating CSV: 		/projects/my-public-bucket/test_output/ATL08_30m_20210714235338_03261206_005_01_30m.csv


### Compile granules of the fails for another DPS batch (Round2)

In [181]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [189]:
import pprint
DPS_INPUT_ATL08_GRANULE_LIST_1, DPS_INPUT_ATL08_GRANULE_LIST_2, DPS_INPUT_ATL08_GRANULE_LIST_3, DPS_INPUT_ATL08_GRANULE_LIST_4, DPS_INPUT_ATL08_GRANULE_LIST_5 = list(chunks(DPS_INPUT_ATL08_GRANULE_LIST, 5000))
print(
    len(DPS_INPUT_ATL08_GRANULE_LIST_1), len(DPS_INPUT_ATL08_GRANULE_LIST_2), len(DPS_INPUT_ATL08_GRANULE_LIST_3), len(DPS_INPUT_ATL08_GRANULE_LIST_4), len(DPS_INPUT_ATL08_GRANULE_LIST_5)
)

5000 5000 5000 5000 280


### Examine output of DPS - but replace glob.glob with s3.glob

In [19]:
#print(f"The data frame show you submitted {len(job_results_df)} jobs. Check the returned results to see if the total returned = total submitted...")
for JOB_HOUR in range(20,21):
    returned_results_list = glob.glob(f"/projects/my-private-bucket/dps_output/run_extract_filter_atl08_ubuntu/master/2022/03/04/{JOB_HOUR}/**/*.csv", recursive=True)
    print(f"For DPS job that returned results in hour {JOB_HOUR}, # granules that ran: {len(returned_results_list)}")

For DPS job that returned results in hour 20, # granules that ran: 10


In [32]:
# Merge all files in the list
print("Creating pandas data frame...")
atl08_gdf = pd.concat((pd.read_csv(f) for f in returned_results_list ), sort=False, ignore_index=True) # <--generator is (), list is []
atl08_gdf = gpd.GeoDataFrame(atl08_gdf, geometry=gpd.points_from_xy(atl08_gdf.lon, atl08_gdf.lat), crs='epsg:4326')

Creating pandas data frame...


In [34]:
atl08_gdf.head()

,fid,lon,lat,dt,orb_orient,orb_num,rgt,gt,segid_beg,segid_end,...,asr,h_dif_ref,ter_flg,ph_rem_flg,dem_rem_flg,seg_wmask,lyr_flg,seg_cover,granule_name,geometry
0,14887,75.620664,45.000490,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249813.842915,249817.842915,...,0.158199,-1.342682,1,0,0,0,0,48,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62066 45.00049)
1,14888,75.620628,45.000758,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249815.336919,249819.336919,...,0.158199,-1.342682,1,0,0,0,0,48,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62063 45.00076)
2,14889,75.620401,45.002371,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249824.331298,249828.331298,...,0.158199,-1.073212,1,0,0,0,0,42,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62040 45.00237)
3,14890,75.620364,45.002640,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249825.830880,249829.830880,...,0.158199,-1.073212,1,0,0,0,0,42,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62036 45.00264)
4,14891,75.620291,45.003178,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249828.830880,249832.830880,...,0.129356,-1.743439,1,0,0,0,0,24,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62029 45.00318)


In [27]:
os.system( "python /projects/icesat2_boreal/lib/build_tindex_master.py -t ATL08 -dps_year 2022 -m 3 --start_day '04' --outdir /projects/my-public-bucket/test_output")

512